In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [36]:
from datetime import datetime as dt

start = dt.now()
df = pd.read_csv('../input/us-accidents/US_Accidents_Dec21_updated.csv')

In [37]:
df.dropna(inplace = True)
df=df[df.Severity != 0]
df.drop(columns = ['ID'], inplace = True)
df.drop(columns = ['Description'], inplace = True)

In [38]:
categorical_columns = df.select_dtypes(include=['object','bool','datetime64[ns]']).columns
categorical_columns
from sklearn import preprocessing

for i in categorical_columns:
    en = preprocessing.LabelEncoder()
    df[i]= en.fit_transform(df[i])

In [39]:
df.head()

,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Number,Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
42,4,0,0,40.45112,-85.15048,40.35429,-85.14993,6.690,9001.0,95429,1,2051,632,13,164117,0,1,1040,0,19.9,7.3,81.0,29.85,2.0,20,12.7,0.00,45,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
43,4,0,0,40.35429,-85.14993,40.45112,-85.15048,6.690,473.0,55877,1,6318,632,13,164165,0,1,1040,0,19.9,7.3,81.0,29.85,2.0,20,12.7,0.00,45,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
47,4,1,1,40.72813,-84.78965,40.74559,-84.78962,1.206,12998.0,84801,1,8311,1304,33,163329,0,1,623,1,17.1,0.6,77.0,29.91,2.5,22,19.6,0.01,45,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0
50,2,2,2,41.83193,-80.10143,41.84149,-80.11099,0.824,25529.0,37994,0,1033,316,36,18858,0,1,665,2,21.0,9.9,85.0,29.69,1.0,21,10.4,0.00,45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
66,2,3,3,41.48339,-81.66297,41.47692,-81.66075,0.462,3937.0,40151,0,1429,333,33,160411,0,1,233,3,24.1,7.1,75.0,29.82,2.0,22,28.8,0.00,45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:
#logistic regression
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc

In [41]:
target = 'Severity'
y = df[target]
X = df.drop(target, axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

In [42]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [43]:
accuracy_lst=[]
lr = LogisticRegression(random_state=0,max_iter=300)
import time
start = time.time()
lr.fit(X_train,y_train)
print("Total time: ", time.time() - start, "seconds")
y_pred=lr.predict(X_test)

# Get the accuracy score
acc=accuracy_score(y_test, y_pred)

# Append to the accuracy list
accuracy_lst.append(acc)
print("[Logistic regression algorithm] accuracy_score: {:.3f}.".format(acc))


Total time:  111.73265242576599 seconds
[Logistic regression algorithm] accuracy_score: 0.944.


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [46]:
from sklearn.decomposition import PCA
pca = PCA(random_state=44)
import time
start = time.time()
pca.fit(X_train)
print("Total time: ", time.time() - start, "seconds")
PCA_pred=lr.predict(X_test)
# Get the accuracy score
acc_pca=accuracy_score(y_test,PCA_pred)
print(acc_pca)

Total time:  1.410473346710205 seconds
0.9439161684264089


In [48]:
from sklearn.metrics import recall_score

rec = recall_score(y_test, y_pred,average=None)

recpca= recall_score(y_test, PCA_pred,average=None)
print(rec)
print(recpca)

[0.         0.99920216 0.         0.04196045]
[0.         0.99920216 0.         0.04196045]
